# 01 — Siamese Tracker Architecture

This notebook explains the Siamese network architecture used for single-object tracking,
visualises the backbone feature maps, and provides a t-SNE projection of learned embeddings.

In [ ]:
import sys
from pathlib import Path

# Ensure project root is on the path
ROOT = Path.cwd().parent  # notebooks/ → mot/
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import numpy as np
import matplotlib.pyplot as plt
from src.training.siamese_model import build_siamese_tracker_model, build_siamese_backbone

## 1. Model Summary
Build the full Siamese tracker and print a summary.

In [ ]:
model = build_siamese_tracker_model()
model.summary()

## 2. Backbone Feature Maps
Feed a random template through the shared backbone and visualise the feature maps.

In [ ]:
backbone = build_siamese_backbone()
sample = np.random.rand(1, 127, 127, 3).astype(np.float32)
features = backbone.predict(sample, verbose=0)
print(f"Feature map shape: {features.shape}")

fig, axes = plt.subplots(4, 8, figsize=(16, 8))
for i, ax in enumerate(axes.flat):
    if i < features.shape[-1]:
        ax.imshow(features[0, :, :, i], cmap='viridis')
    ax.axis('off')
fig.suptitle('Backbone Feature Maps (first 32 channels)', fontsize=14)
plt.tight_layout()
plt.show()

## 3. Response Map Visualisation
Feed a template+search pair and visualise the cross-correlation response map.

In [ ]:
t = np.random.rand(1, 127, 127, 3).astype(np.float32)
s = np.random.rand(1, 255, 255, 3).astype(np.float32)
resp = model.predict([t, s], verbose=0)
print(f"Response map shape: {resp.shape}")

plt.figure(figsize=(6, 5))
plt.imshow(resp[0, :, :, 0], cmap='hot')
plt.colorbar(label='Score')
plt.title('Cross-Correlation Response Map')
plt.show()

## 4. t-SNE of Backbone Embeddings
Generate embeddings for several patches and project them to 2-D with t-SNE.

In [ ]:
from sklearn.manifold import TSNE

n_samples = 200
patches = np.random.rand(n_samples, 127, 127, 3).astype(np.float32)
feats = backbone.predict(patches, verbose=0)
flat = feats.reshape(n_samples, -1)

tsne = TSNE(n_components=2, perplexity=30, random_state=42)
proj = tsne.fit_transform(flat)

plt.figure(figsize=(8, 6))
plt.scatter(proj[:, 0], proj[:, 1], s=8, alpha=0.7)
plt.title('t-SNE of Backbone Embeddings (random patches)')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.tight_layout()
plt.show()